In [6]:
import os
import requests
import json
from dotenv import load_dotenv

In [7]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
personal_api_key = os.getenv("PERSONAL_API_KEY")

In [8]:
#create json with data
data = []
files_names = []
for file in os.listdir("../pliki_z_fabryki/do-not-share"):
    files_names.append(file)
    with open(f"../pliki_z_fabryki/do-not-share/{file}", "rb") as f:
        text = f.read()
        text = text.decode("utf-8")
        text = text.replace("\n", " ")
        data.append( {"name": file, "text": text})
print(files_names)
print(json.dumps(data, indent=4, ensure_ascii=False))

['2024_04_27.txt', '2024_01_08.txt', '2024_07_05.txt', '2024_05_08.txt', '2024_01_17.txt', '2024_02_01.txt', '2024_02_15.txt', '2024_03_19.txt', '2024_03_12.txt', '2024_06_02.txt', '2024_03_15.txt', '2024_03_25.txt', '2024_01_29.txt', '2024_02_21.txt', '2024_05_14.txt', '2024_01_27.txt', '2024_05_31.txt', '2024_03_31.txt', '2024_03_18.txt', '2024_02_11.txt', '2024_04_18.txt', '2024_03_02.txt', '2024_03_29.txt']
[
    {
        "name": "2024_04_27.txt",
        "text": "Wysokotemperaturowy Pojemnik Kinetyczny (WPK)  Raport z testów Wysokotemperaturowego Pojemnika Kinetycznego wykazał, że konstrukcja pozwala na generowanie ekstremalnych temperatur przy jednoczesnym wydobywaniu masywnej energii kinetycznej. Podczas testów osiągnięto temperatury przekraczające 3000 stopni Celsjusza, co skutkowało efektywnym stopieniem materiałów o wysokiej twardości, takich jak stal.  Wyniki testów sugerują, że WPK może być użyty zarówno do niszczenia wrogich pojazdów, jak i do przekształcania terenu w ter

In [35]:
from OpenAIService import OpenAIService
from VectorService import VectorService
from TextService import TextSplitter

openai = OpenAIService();
vector_service = VectorService(openai);
text_splitter = TextSplitter();


In [104]:
COLLECTION_NAME = "s03e02"
async def initialize_data(openai: OpenAIService, 
                        vector_service: VectorService, 
                        text_splitter: TextSplitter) -> None:
    points = []
    for item in data:
        doc = await text_splitter.document(item["text"], "gpt-4o", {"name": item["name"]})
        points.append(doc)
    print(json.dumps(points, indent=4, ensure_ascii=False))
    await vector_service.initialize_collection_with_data(COLLECTION_NAME, points)

In [105]:
await initialize_data(openai, vector_service, text_splitter)

[
    {
        "text": "Wysokotemperaturowy Pojemnik Kinetyczny (WPK)  Raport z testów Wysokotemperaturowego Pojemnika Kinetycznego wykazał, że konstrukcja pozwala na generowanie ekstremalnych temperatur przy jednoczesnym wydobywaniu masywnej energii kinetycznej. Podczas testów osiągnięto temperatury przekraczające 3000 stopni Celsjusza, co skutkowało efektywnym stopieniem materiałów o wysokiej twardości, takich jak stal.  Wyniki testów sugerują, że WPK może być użyty zarówno do niszczenia wrogich pojazdów, jak i do przekształcania terenu w teren nieprzyjazny dla przeciwnika. Wnioski wskazują na potrzebę dalszego rozwijania pojemnika, aby poprawić jego trwałość i efektywność w zmiennych warunkach atmosferycznych.  Roboty badawcze planują przeprowadzić dodatkowe testy w ekstremalnych temperaturach oraz modyfikacje konstrukcji, aby zwiększyć portfel zastosowań WPK w wojskowych operacjach postapokaliptycznych. ",
        "metadata": {
            "tokens": 351,
            "headers": {},

In [106]:
query = "Kradzież prototypu, kradzież broni"
determine_raport_date = openai.completion(  
    messages=[
        {"role": "system", "content": f"""You are a helpful assistant that determines the name of the raport.
                                        Pick between {files_names}. just with name. If both are relevant, list them comma-separated. Write back with the name(s) and nothing else."""},
        {"role": "user", "content": query}
    ]
)   
print(determine_raport_date)
raportNames = determine_raport_date.choices[0].message.content.split(',') if determine_raport_date.choices[0].message.content else []
raportNames = [raportName.strip() for raportName in raportNames]
print(raportNames)

ChatCompletion(id='chatcmpl-AWgJs5oIadY1oqsD7bROZkG6wpZgl', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2024_01_29.txt', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732352536, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_7f6be3efb0', usage=CompletionUsage(completion_tokens=7, prompt_tokens=273, total_tokens=280, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
['2024_01_29.txt']


In [111]:
# Create filter based on authors
filter_dict = {
    "should": [
        {
            "key": "name",
            "match": {"value": raportName}
        } for raportName in files_names
    ]
} if raportNames else None

# Perform search
search_results = await vector_service.perform_search(COLLECTION_NAME, query, filter_dict, 3)
print(search_results)

[ScoredPoint(id='e4fd5455-0c58-4317-88f0-30fa6e501e1a', version=0, score=0.49812376, payload={'text': 'Prototyp podręcznego zakrzywiacza czasoprzestrzeni  Niniejszy raport odnosi się do incydentu związanego z kradzieżą prototypu podręcznego zakrzywiacza czasoprzestrzeni, który miał na celu umożliwienie skoków w czasie. Z przeprowadzonych testów wynika, że urządzenie działało zgodnie z założeniami, jednak jego efektywnie ograniczone były przez stabilność zakrzywienia, co prowadziło do nieprzewidywalnych konsekwencji w czasie i przestrzeni.  Urządzenie umożliwia dokonywanie krótkich przeskoków w czasie i przestrzeni, jednak jego stosunkowo duże zużycie energii wyklucza go z profesjonalnego zastosowania.  Wnioski wyciągnięte z testów wskazują na konieczność dalszych prac nad poprawą stabilności oraz precyzji urządzenia. Dodatkowo, incydent kradzieży ujawnił poważne braki w procedurach zabezpieczających prototypy, co wymaga natychmiastowego przeglądu obecnych zabezpieczeń.  Plany na przysz

In [112]:
# Check relevance
relevance_checks = []
for result in search_results:
    content = f"Query: {query}\nText: {result.payload.get('text', '')}"
    print(content)
    relevance_check = openai.completion(
        messages=[
            {"role": "system", "content": "You are a helpful assistant that determines if a given text is relevant to a query. Respond with 1 if relevant, 0 if not relevant."},
            {"role": "user", "content": content}
        ]
    )
    print(relevance_check)
    is_relevant = relevance_check.choices[0].message.content == "1"
    result_dict = dict(result)
    result_dict["is_relevant"] = is_relevant
    relevance_checks.append(result_dict)

relevant_results = [result for result in relevance_checks if result["is_relevant"]]
print(relevant_results)

Query: Kradzież prototypu, kradzież broni
Text: Prototyp podręcznego zakrzywiacza czasoprzestrzeni  Niniejszy raport odnosi się do incydentu związanego z kradzieżą prototypu podręcznego zakrzywiacza czasoprzestrzeni, który miał na celu umożliwienie skoków w czasie. Z przeprowadzonych testów wynika, że urządzenie działało zgodnie z założeniami, jednak jego efektywnie ograniczone były przez stabilność zakrzywienia, co prowadziło do nieprzewidywalnych konsekwencji w czasie i przestrzeni.  Urządzenie umożliwia dokonywanie krótkich przeskoków w czasie i przestrzeni, jednak jego stosunkowo duże zużycie energii wyklucza go z profesjonalnego zastosowania.  Wnioski wyciągnięte z testów wskazują na konieczność dalszych prac nad poprawą stabilności oraz precyzji urządzenia. Dodatkowo, incydent kradzieży ujawnił poważne braki w procedurach zabezpieczających prototypy, co wymaga natychmiastowego przeglądu obecnych zabezpieczeń.  Plany na przyszłość obejmują nie tylko prace nad poprawą zakrzywiacza,

In [121]:
# Print results
answer = relevant_results[0]["payload"]["name"].replace("_", "-").replace(".txt", "")
print(f"Query: {query}")
# Create table-like output
print("\n{:<20} {:<50} {:<10}".format("name", "text", "score"))
print("-" * 80)
for result in relevant_results:
    name = result["payload"]["name"].replace("_", "-").replace(".txt", "")
    text = result["payload"]["text"][:47] + "..." if len(result["payload"]["text"]) > 50 else result["payload"]["text"]
    text = text.replace("\n", " ")
    score = f"{result['score']:.3f}"
    print("{:<20} {:<50} {:<10}".format(name, text, score))

Query: Kradzież prototypu, kradzież broni
raportName(s): 2024-02-21

name                 text                                               score     
--------------------------------------------------------------------------------


In [122]:
answer_json = {
    "task": "wektory",
    "apikey": personal_api_key,
    "answer": answer
}
answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=answer_json)
print(answer_response.request.body)  # Displays the request body

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8'))       # Displays the response content as a string    

b'{"task": "wektory", "apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": "2024-02-21"}'
Status Code: 200
Response Body: b'{\n    "code": 0,\n    "message": "{{FLG:ZLODZIEJ}}"\n}'
